In [1]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import shap
from xgboost import XGBClassifier
# import random forest regressor
from sklearn.ensemble import RandomForestRegressor
#import linear regression
from sklearn.linear_model import LinearRegression
# import tqdm
from tqdm import tqdm
import tqdm
#import r2_score
from sklearn.metrics import r2_score
#import confusion matrix
from sklearn.metrics import confusion_matrix
# import roc auc score
from sklearn.metrics import roc_auc_score
from food_crisis_functions import *
import json
with open("forecasting_hyperparameters.json", "r") as file:
    best_params_xgb_regressor= json.load(file)
    
with open("forecasting_hyperparameters_p3.json", "r") as file:
    best_params_xgb_regressor_for_p3= json.load(file)
# read csv
df = pd.read_csv(r'C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\forecasting_subset_IPCCH_v1210.csv')
# add dummys for area_id and month year
#df = pd.concat([df, pd.get_dummies(df['area_id'], prefix='area_id')], axis=1)
#df = pd.concat([df, pd.get_dummies(df['date'], prefix='month_year')], axis=1)
# drop lat and lon
#df = df.drop(['lat', 'lon'], axis=1)
###drop fews_ipc_ha
#df = df.drop(['fews_ipc_ha'], axis=1)
# random split train and test

# prepare date from year and month
df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY=1))
# check for inf and replace with na
df = df.replace([np.inf, -np.inf], np.nan)

# replace df['overall_phase], 0 as 1, >5 as 5
df['overall_phase'] = df['overall_phase'].apply(lambda x: 1 if x < 1 else (5 if x > 5 else x))

df_origin = df.copy()
y_pred_test = pd.DataFrame()
model_stats = pd.DataFrame()
#select phase1_percent is not na
df = df_origin[df_origin['phase1_percent'].notna()]

# Sort by region and date
df = df.sort_values(by=['area_id', 'date'])
#drop overall phase
df = df.drop(['overall_phase'], axis=1)
#for each region, set last observation to be test set
# create a series of new outcome, phase2_worse=phase2_percent+phase3_percent+phase4_percent+phase5_percent, phase3_worse=phase3_percent+phase4_percent+phase5_percent, phase4_worse=phase4_percent+phase5_percent, phase5_worse=phase5_percent
df['phase2_worse'] = df['phase2_percent'] + df['phase3_percent'] + df['phase4_percent'] + df['phase5_percent']
df['phase3_worse'] = df['phase3_percent'] + df['phase4_percent'] + df['phase5_percent']
df['phase4_worse'] = df['phase4_percent'] + df['phase5_percent']
df['phase5_worse'] = df['phase5_percent']
#drop phase2_percent, phase3_percent, phase4_percent, phase5_percent, phase1_percent
df = df.drop(['phase2_percent', 'phase3_percent', 'phase4_percent', 'phase5_percent', 'phase1_percent'], axis=1)
# Splitting the data
#test_df = df.groupby('area_id').tail(1)
#train_df = df.drop(test_df.index)
#test_df = test_df.drop(['area_id','date'], axis=1)
#train_df = train_df.drop(['area_id','date'], axis=1)


# Splitting the data
#test_df = df.groupby('area_id').tail(1)
#train_df = df.drop(test_df.index)
#test_df = test_df.drop(['area_id','date'], axis=1)
#train_df = train_df.drop(['area_id','date'], axis=1)
y_pred_test = pd.DataFrame()
# drop anything after 2022-01-01
#df = df[df['date'] < '2021-01-01']
shape_values_df_ensemble = pd.DataFrame()
df_result = pd.DataFrame()
date = "2022-01-01"  # Define the 'date' variable
cutoff_date = "2023-01-01"
#order unique_dates
y_pred_test=pd.DataFrame()

for i in range(2, 6):
    train_df = df[df['date'] < date]
    test_df = df[(df['date'] < cutoff_date) & (df['date'] >= date)]
    train_df = train_df.drop(['date','area_id'], axis=1)
    test_df = test_df.drop(['date','area_id'], axis=1)
    train_df_new = train_df.drop(['phase{}_worse'.format(j) for j in range(2, 6) if j != i], axis=1)
    test_df_new = test_df.drop(['phase{}_worse'.format(j) for j in range(2, 6) if j != i], axis=1)
# drop rows with NaN in phase{}_percent
    train_df_new = train_df_new.dropna(subset=['phase{}_worse'.format(i)])
    test_df_new = test_df_new.dropna(subset=['phase{}_worse'.format(i)])
    test_index = test_df_new.index
    # Split into features and target
    X_train = train_df_new.drop('phase{}_worse'.format(i), axis=1)
    y_train = train_df_new['phase{}_worse'.format(i)]
    X_test = test_df_new.drop('phase{}_worse'.format(i), axis=1)
    y_test = test_df_new['phase{}_worse'.format(i)]
    #fews_ipc_ha_test = X_test['fews_ipc_ha']
    #X_train = X_train.drop(['fews_ipc_ha'], axis=1)
    #X_test = X_test.drop(['fews_ipc_ha'], axis=1)
    if i == 3:
        best_params_xgb_regressor = best_params_xgb_regressor_for_p3
    model = xgb.XGBRegressor(**best_params_xgb_regressor)
    model.fit(X_train, y_train)
    # Predictions
    y_pred = model.predict(X_test)
    # for y_pred_test, add a column to indicate the phase
    #y_pred_test = pd.concat([y_pred_test, pd.DataFrame({'y_pred': y_pred, 'y_test': y_test, 'phase': [i]*len(y_pred),'fews_ipc_ha':fews_ipc_ha_test,'test_index':test_index})], ignore_index=True)
    y_pred_test = pd.concat([y_pred_test, pd.DataFrame({'y_pred': y_pred, 'y_test': y_test, 'phase': [i]*len(y_pred),'test_index':test_index})], ignore_index=True)
   
    # shap values
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_train)
    
    #save shap values
    shap_values_df = pd.DataFrame(shap_values, columns=X_train.columns)
    
    # add a column to indicate the phase
    shap_values_df['phase'] = i
    
    # append to shape_values_df_ensemble
    shape_values_df_ensemble = pd.concat([shape_values_df_ensemble, shap_values_df], ignore_index=True)
y_pred_test = convert_prob_to_phase(y_pred_test)
y_test = y_pred_test['overall_phase']
y_pred = y_pred_test['overall_phase_pred']

test_df = df[(df['date'] < cutoff_date) & (df['date'] >= date)]
#generate month, year in test_df
test_df['month'] = pd.DatetimeIndex(test_df['date']).month
test_df['year'] = pd.DatetimeIndex(test_df['date']).year
# keep only the last three columns
y_pred_test_new = y_pred_test.iloc[:,-3:]
#select only area_id, month, year in test_df
test_df = test_df[['area_id', 'month', 'year']]
#test_df reset index
test_df = test_df.reset_index(drop=True)
# concate horizontally test_df and y_pred_test_new, ignore index
df_result = pd.concat([test_df, y_pred_test_new], axis=1, ignore_index=True)
# rename columns
df_result.columns = ['area_id', 'month', 'year', 'test_index','overall_phase','overall_phase_pred']
# use month and year to create date
df_result['date'] = pd.to_datetime(df_result[['year', 'month']].assign(day=1))
#drop month and year
df_result = df_result.drop(['month', 'year'], axis=1)
# select date, area_id, overall_phase in df
df = pd.read_csv(r'C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\forecasting_subset_IPCCH_v1210.csv')


C:\Users\swl00\AppData\Local\Temp\ipykernel_9964\1679473219.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['month'] = pd.DatetimeIndex(test_df['date']).month
C:\Users\swl00\AppData\Local\Temp\ipykernel_9964\1679473219.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['year'] = pd.DatetimeIndex(test_df['date']).year


In [2]:

df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY=1))
df = df[['date', 'area_id', 'overall_phase']]
# convert date to datetime
df['date'] = pd.to_datetime(df['date'])
# df_result drop test_index
df_result = df_result.drop(['test_index'], axis=1)
# df add a column overall_phase_pred, set it equal to overall_phase
df['overall_phase_pred'] = df['overall_phase']
# df drop any row after 2024-01-01
df = df[df['date'] < '2022-01-01']
# concate df and df_result
df = pd.concat([df, df_result], ignore_index=True)
# add a new column count the nth time the area_id appears
df['count'] = df.groupby('area_id').cumcount() + 1
# groupby area_id, keep rows with two largest count
df = df.groupby('area_id').apply(lambda x: x.nlargest(2, 'count')).reset_index(drop=True)
# set all count==1 to 0, set all count>1 ro 1
df['count'] = df['count'].apply(lambda x: 0 if x == 1 else 1)
#group by area_id, if overall_phase of count==1 is the same as overall_phase of count==0, drop all rows associated with area_id,but if there is only one row associated with area_id, keep it
df_1 = df.groupby('area_id').filter(lambda x: x['overall_phase'].nunique() > 1)
df_2 = df.groupby('area_id').filter(lambda x: x['area_id'].count() == 1)
# concate df_1 and df_2, sort by date and area_id
df = pd.concat([df_1, df_2], ignore_index=True)
df = df.sort_values(by=['date', 'area_id'])
# for each area_id, keep only the last row
df = df.groupby('area_id').tail(1)
#drop count
df = df.drop(['count'], axis=1)
# drop any row before 2024-01-01
df_after = df[(df['date'] < cutoff_date) & (df['date'] >= date)]
#calculate accuracy score using overall_phase and overall_phase_pred


C:\Users\swl00\AppData\Local\Temp\ipykernel_9964\1861004737.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('area_id').apply(lambda x: x.nlargest(2, 'count')).reset_index(drop=True)


In [3]:

#drop missing in df_after
df_after = df_after.dropna(subset=['overall_phase', 'overall_phase_pred'])
accuracy = accuracy_score(df_after['overall_phase'], df_after['overall_phase_pred'])
# set phase 1 and phase 2 as negative, phase 3, 4, 5 as positive, calculate sensitivity(recall) and presicion
cm = confusion_matrix(df_after['overall_phase'], df_after['overall_phase_pred'])
correct_3_more = np.sum(cm[2:, 2:])
total_3_more = np.sum(cm[2:, :])
sensitivity = correct_3_more / total_3_more if total_3_more != 0 else 0

total_3_more_pred = np.sum(cm[:, 2:])
precision = correct_3_more / total_3_more_pred if total_3_more_pred != 0 else 0
accuracy,sensitivity, precision

(0.4955223880597015, 0.5758835758835759, 0.647196261682243)